In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from adjustText import adjust_text

# Initialize a DataFrame to store log2 and log10 values for all samples
log2_log10_results = pd.DataFrame()

# Step 2: Loop over unique sample names
sample_names = CRISPRi['Sample'].unique()

# Debugging: Check all sample names present
print(f"Sample Names in CRISPRi Data: {sample_names}")

for sample in sample_names:
    # Filter data for the current sample
    CRISPRi_sample = CRISPRi[CRISPRi['Sample'] == sample]
    knockouts_sample = knockouts[knockouts['Sample'] == sample]

    # Debugging: Ensure that the sample exists in both datasets
    if sample not in CRISPRi['Sample'].values or sample not in knockouts['Sample'].values:
        print(f"Warning: Sample {sample} is missing in one or both datasets.")
        continue  # Skip the missing sample and move to the next one

    # Step 3: Compute mean %_of protein_abundance_Top3-method for each protein
    CRISPRi_mean = CRISPRi_sample.groupby('Protein.Group')['%_of protein_abundance_Top3-method'].mean().reset_index()
    knockouts_mean = knockouts_sample.groupby('Protein.Group')['%_of protein_abundance_Top3-method'].mean().reset_index()

    # Rename columns for clarity
    CRISPRi_mean.rename(columns={'%_of protein_abundance_Top3-method': 'CRISPRi_abundance'}, inplace=True)
    knockouts_mean.rename(columns={'%_of protein_abundance_Top3-method': 'knockouts_abundance'}, inplace=True)

    # Step 4: Merge the data on Protein.Group
    merged_data = pd.merge(CRISPRi_mean, knockouts_mean, on='Protein.Group', how='outer')  # Ensure all proteins are included

    # Debugging: Print the merged data to check if the sample is properly merged
    print(f"Merged data for sample {sample}: {merged_data.shape} rows")

    # Step 5: Calculate log2 fold change (fill missing values with NaN)
    merged_data['log2_change'] = np.log2(merged_data['knockouts_abundance'] / merged_data['CRISPRi_abundance'])

    # Step 6: Compute p-values (assume replicates data exists for actual p-value calculation)
    CRISPRi_reps = CRISPRi_sample[['Protein.Group', '%_of protein_abundance_Top3-method']]
    knockouts_reps = knockouts_sample[['Protein.Group', '%_of protein_abundance_Top3-method']]

    p_values = []
    for protein in merged_data['Protein.Group']:
        group1 = CRISPRi_reps[CRISPRi_reps['Protein.Group'] == protein]['%_of protein_abundance_Top3-method']
        group2 = knockouts_reps[knockouts_reps['Protein.Group'] == protein]['%_of protein_abundance_Top3-method']
        if not group1.empty and not group2.empty:
            _, p_val = ttest_ind(group1, group2, equal_var=False)
        else:
            p_val = np.nan  # Assign NaN if data is missing
        p_values.append(p_val)

    # Add p-values and log10 transformation
    merged_data['p_value'] = p_values
    merged_data['log10_p_value'] = -np.log10(merged_data['p_value'])

    # Step 7: Ensure consistent row alignment in log2_log10_results
    if log2_log10_results.empty:
        log2_log10_results['Locus'] = merged_data['Protein.Group']  # Start with the first set of loci
    else:
        log2_log10_results = pd.merge(
            log2_log10_results,
            merged_data[['Protein.Group', 'log2_change', 'log10_p_value']],
            left_on='Locus',
            right_on='Protein.Group',
            how='outer'
        ).drop(columns=['Protein.Group'])

    # Add current sample's log2 and log10 values as new columns
    log2_log10_results.rename(
        columns={'log2_change': f'{sample}_log2_change', 'log10_p_value': f'{sample}_log10_p_value'},
        inplace=True
    )

    # Step 8: Create volcano plot for the current sample
    fold_change = merged_data['log2_change']
    p_values = merged_data['log10_p_value']
    log_sig_thresh = 2

    plt.figure(figsize=(8, 6))
    plt.scatter(fold_change, p_values, s=15, alpha=0.25)  # Adjust marker size and transparency as needed
    plt.title(f"Knockouts/CRISPRi ({sample})")  # Title reflects the sample name
    plt.xlabel("Log2(Fold Change)")
    plt.ylabel("Log10(P-Value)")
    plt.grid(False)
    plt.ylim(0,)  # Adjust y-axis range if needed
    plt.xlim(-10, 10)  # Adjust x-axis range if needed

    # Threshold lines for fold change and p-values
    plt.axvline(x=log_sig_thresh, color='r', linestyle='--', linewidth=1)  # Fold change > 2
    plt.axvline(x=-log_sig_thresh, color='r', linestyle='--', linewidth=1)  # Fold change < -2
    plt.axhline(y=-np.log10(0.01), color='g', linestyle='--', linewidth=1)  # P-value < 0.01

    # Annotate points that meet specific conditions
    texts = []
    labels = merged_data['Protein.Group']  # Use translated locus names for annotation
    for i, label in enumerate(labels):
        if fold_change[i] < -log_sig_thresh and p_values[i] > -np.log10(0.01):  # Modify conditions as necessary
            text = plt.text(fold_change[i], p_values[i], label, fontsize=10)
            texts.append(text)

    # Adjust text annotations to avoid overlaps
    if texts:
        adjust_text(texts, arrowprops=dict(arrowstyle='-', color='red', lw=1))

    # Show the plot
    plt.show()

# Step 9: Save the log2 and log10 values to a CSV file
log2_log10_results.to_csv('log2_log10_values_by_sample.csv', index=False)
print("Log2 and Log10 values saved to 'log2_log10_values_by_sample.csv'")


NameError: name 'CRISPRi' is not defined